# Datos abiertos de la CDMX

## Atención:
Todos los gráficos del siguiente reporte son interactivos mediante el cliqueo de la leyenda

## Optención y exploración de datos

In [1]:
# importe de datos en csv
import pandas as pd
datos=pd.read_csv("datos.csv")

In [2]:
# previsualizacion de las primeras lineas de datos
datos.head().T

,0,1,2,3,4
_id,1,2,3,4,5
ao_hechos,2016,2016,2016,2016,2015
mes_hechos,Enero,Enero,Febrero,Enero,Diciembre
fecha_hechos,2016-01-31 22:16:00,2016-01-31 20:50:00,2016-02-01 00:30:00,2016-01-31 22:00:00,2015-12-25 12:00:00
ao_inicio,2016,2016,2016,2016,2016
mes_inicio,Febrero,Febrero,Febrero,Febrero,Febrero
fecha_inicio,2016-02-01T00:25:44,2016-02-01T00:52:37,2016-02-01T01:33:26,2016-02-01T02:09:11,2016-02-01T02:16:49
delito,DAÑO EN PROPIEDAD AJENA INTENCIONAL,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,NARCOMENUDEO POSESION SIMPLE,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,DENUNCIA DE HECHOS
fiscalia,INVESTIGACIÓN EN BENITO JUÁREZ,INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE R...,"INVESTIGACIÓN PARA LA ATENCIÓN DE NIÑOS, NIÑAS...",INVESTIGACIÓN EN IZTAPALAPA,INVESTIGACIÓN EN BENITO JUÁREZ
agencia,BJ-2,ORIENTEII,57,IZP-6,BJ-1


## Datos inválidos

Se observa para cada columna, la cantidad de datos inválidos que contiene y el porcentaje que representa.

In [3]:
inv=datos.isna().sum().sort_values(ascending=False)
pd.DataFrame({"Nombre de columna":inv.index,"N":inv.values,"%":inv.values/len(datos)})

,Nombre de columna,N,%
0,tempo,209455,1.000000
1,competencia,209455,1.000000
2,calle_hechos2,118600,0.566231
3,colonia_hechos,8382,0.040018
4,longitud,8155,0.038934
5,latitud,8155,0.038934
6,calle_hechos,621,0.002965
7,alcaldia_hechos,399,0.001905
8,unidad_investigacion,122,0.000582
9,ao_hechos,74,0.000353


### Comentario:

Algunos datos como "calle de los hechos2" presenta mas de 50% de datos perdidos. Es importante ver si los datos perdidos pueden afectar el desarrollo de la investigación. Si no, pueden simplemente descartarse. En este caso las columnas con mas datos perdidos no parecen ser necesarios para este estudio.

### Fechas de estudio

Es importante investigar que años se estan estudiando, y si se incluyen todos los meses del año.

In [4]:
# convierte la fecha en datetime
datos['fecha_hechos'] = pd.to_datetime(datos['fecha_hechos'],infer_datetime_format=True)
datos['año_hechos'] = pd.DatetimeIndex(datos['fecha_hechos']).year

In [5]:
# que años hay en los datos? cual es el minimo y maximo?
{"años":len(datos['año_hechos'].dropna().unique()),"min_año":min(datos['año_hechos'].dropna().unique()),"max_año":max(datos['año_hechos'].dropna().unique()),"n_meses":len(datos.mes_hechos.dropna().unique())}

{'años': 47, 'max_año': 2019.0, 'min_año': 1915.0, 'n_meses': 12}

## Cuales son los delitos con mayor incidencia?

In [6]:
import numpy as np

delitos = datos.groupby("delito",as_index=False).count()[["delito","_id"]].sort_values(by = "_id",ascending=False)
delitos.columns = ["delito","N"]

print({"porcentaje acumulado":np.sum((delitos.N/np.sum(delitos.N))[0:30])})
print(delitos[0:30])

{'porcentaje acumulado': 0.7837148790909741}
                                                delito      N
276                                 VIOLENCIA FAMILIAR  18532
228                                    ROBO DE OBJETOS  15097
161                       ROBO A NEGOCIO SIN VIOLENCIA  14022
50                                  DENUNCIA DE HECHOS  13552
77                                              FRAUDE  11662
9                                             AMENAZAS   9788
210     ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA   7365
157               ROBO A CASA HABITACION SIN VIOLENCIA   5954
238  ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...   5578
230        ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO   5370
217                         ROBO DE ACCESORIOS DE AUTO   5276
73   FALSIFICACION DE TITULOS AL PORTADOR Y DOCUMEN...   5193
201         ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA   5117
202         ROBO A TRANSEUNTE DE CELULAR SIN VIOLENCIA   4156
27   DAÑO EN PROPIEDAD AJ

### Comentario:
 Casi el 80% de los delitos corresponden a los primeros 30 tipos de delitos 

### Gráfico de Barras que muestra los delitos mas importantes debido a su incidencia

In [7]:
# grafico de barras de la incidencia de los 30 delitos predominantes

import plotly.express as px
fig = px.bar(delitos[0:30], x="N", y="delito", orientation='h',title="Delitos vs incidencia")
fig.show()


### Comentario:

El delito de mayor incidencia a través del tiempo de estudio es la violencia familiar

## Delitos en alza o en baja ?

Para este estudio de comportamiento de delitos hemos seleccionado los 30 delitos de mayor incidencia y los datos para este milenio (del año 2000 en adelante) 

In [12]:
import plotly.graph_objects as go

title = 'Incidencia de delitos'
labels = datos["delito"].unique()

# se filtran los datos para los 30 delitos mas importantes
data = datos[datos.delito.isin(delitos.delito[0:30])]
# se convierte la columna de años en categorica
data.loc[:,"año_hechos"] = pd.Categorical(data["año_hechos"], categories = data["año_hechos"].dropna().unique())
# se agrupan los datos por año y delito
data = data.groupby(["año_hechos","delito"], as_index=False).count()[["año_hechos","delito","_id"]]
# se llenan los valores na con 0
data.loc[:,"N"] = data["_id"].fillna(0)
# se toman las columnas necesarias
data = data[["año_hechos","delito","N"]]

data.año_hechos=np.array(data.año_hechos,dtype="int")
fig = go.Figure()

for i in range(len(data["delito"].unique())):
    datai=data[data.delito==labels[i]].sort_values(by="año_hechos")
    fig.add_trace(go.Scatter(x=datai.año_hechos, y=datai.N,
        name=labels[i]
    ))

fig.update_layout(title=title)

fig.show()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### COMENTARIO:
Los ultimos años de la muestra parecen ser los mas interesantes, por eso solo miraremos esa parte (2010 en adelante) para nuestro estudio.

In [17]:
# se filtran los datos para los 30 delitos mas importantes
delito_group = datos[datos.delito.isin(delitos.delito[0:30])]
# delitos que ocurrieron despues del año 2000
delito_group = delito_group[delito_group.año_hechos>=2010]
# se convierte la columna de años en categorica
delito_group.loc[:,"año_hechos"] = pd.Categorical(delito_group["año_hechos"], categories=delito_group["año_hechos"].dropna().unique())
# se agrupan los datos por año y delito
delito_group = delito_group.groupby(["año_hechos","delito"], as_index=False).count()[["año_hechos","delito","_id"]]
# se llenan los valores na con 0
delito_group.loc[:,"N"] = delito_group["_id"].fillna(0)
# se toman las columnas necesarias
delito_group = delito_group[["año_hechos","delito","N"]]

### ATENCIÓN:

En los gráficos siguientes selecciona el delito de interés en la leyenda para poder ver la curva.


### Incidencia de delitos desde el año 2010


In [18]:
import plotly.graph_objects as go

title = 'Incidencia de delitos desde el año 2010'
labels = delito_group["delito"].unique()

data = delito_group
data.año_hechos=np.array(data.año_hechos,dtype="int")
fig = go.Figure()

for i in range(len(delito_group["delito"].unique())):
    datai=data[data.delito==labels[i]].sort_values(by="año_hechos")
    fig.add_trace(go.Scatter(x=datai.año_hechos, y=datai.N,
        name=labels[i],
        visible = "legendonly"
    ))
fig.update_layout(title=title)

fig.show()

### Estudio de la tendencia en para los delitos

In [19]:
def derivada(x):
    d=[]
    for i in range(1,len(x)-1):
       d.append(x[i+1]-x[i-1])
    return d

fig = go.Figure()
title = 'Cambio en incidencia de delitos desde el año 2000'
for i in range(len(delito_group["delito"].unique())):
    datai=data[data.delito==labels[i]].sort_values(by="año_hechos")
    datai=datai.reset_index()
    x=datai.año_hechos
    datai=derivada(datai.N)
    fig.add_trace(go.Scatter(x=x, y=datai,
        name=labels[i],
        visible = "legendonly"
    ))
fig.update_layout(title=title)

fig.show()

### COMENTARIO:
Para todos los delitos del estudio se observa que los delitos tienen una tendencia a la baja (pendiente negativa en el gráfico anterior) en los últimos años. Algunos tienen mas pendiente que otros, pero la tendencia general es a la disminución de delitos.

## Estudio de tendencia estacional de los datos

Se quiere investigar si los datos presentan un comportamiento estacionario

In [22]:
datos['mes_hechos'] = pd.DatetimeIndex(datos['fecha_hechos']).month
datos['dia_hechos'] = pd.DatetimeIndex(datos['fecha_hechos']).day
datos['hora_hechos'] = pd.DatetimeIndex(datos['fecha_hechos']).hour
datos['diasem_hechos'] = pd.DatetimeIndex(datos['fecha_hechos']).weekday
# se filtran los datos para los 30 delitos mas importantes
datos = datos[datos.delito.isin(delitos.delito[0:30])]
# delitos que ocurrieron despues del año 2000
datos = datos[datos.año_hechos>=2010]

### Tendencia mensual

In [28]:
title = 'Incidencia general de delitos por mes desde el año 2010'
labels = delito_group["delito"].unique()

data = datos

fig = go.Figure()
data=data.groupby("mes_hechos",as_index=False).count().sort_values(by="mes_hechos")
fig.add_trace(go.Scatter(x=data.mes_hechos, y=data._id))
fig.update_layout(title=title)

fig.show()

In [27]:
title = 'Incidencia de delitos por mes desde el año 2010'
labels = delito_group["delito"].unique()

data = datos

fig = go.Figure()

for i in range(len(delito_group["delito"].unique())):
    datai=data[data.delito==labels[i]].groupby("mes_hechos",as_index=False).count().sort_values(by="mes_hechos")
    fig.add_trace(go.Scatter(x=datai.mes_hechos, y=datai._id,
        name=labels[i],
        visible = "legendonly"
    ))
fig.update_layout(title=title)

fig.show()

### COMENTARIO:

Se observa un comportamiento estacionario mensual

### Comportamiento por dia del mes

In [30]:
title = 'Incidencia general de delitos por dia del mes desde el año 2010'
labels = delito_group["delito"].unique()

data = datos

fig = go.Figure()

data=data.groupby("dia_hechos",as_index=False).count().sort_values(by="dia_hechos")
fig.add_trace(go.Scatter(x=data.dia_hechos, y=data._id,
))
fig.update_layout(title=title)

fig.show()

In [32]:
title = 'Incidencia de delitos por dia del mes desde el año 2010'
labels = delito_group["delito"].unique()

data = datos

fig = go.Figure()

for i in range(len(delito_group["delito"].unique())):
    datai=data[data.delito==labels[i]].groupby("dia_hechos",as_index=False).count().sort_values(by="dia_hechos")
    fig.add_trace(go.Scatter(x=datai.dia_hechos, y=datai._id,
        name=labels[i],
        visible = "legendonly"
    ))
fig.update_layout(title=title)

fig.show()

### COMENTARIO:
No se observa ningún, patrón por dia del mes. Solamente una disminución los 31, pero esto se puede explicar porque no todos los meses llegan hasta el 31.

### Comportamiento por dia de la semana

In [33]:
title = 'Incidencia general de delitos por dia de la semana desde el año 2010'
labels = delito_group["delito"].unique()

data = datos

fig = go.Figure()

data=data.groupby("diasem_hechos",as_index=False).count().sort_values(by="diasem_hechos")
fig.add_trace(go.Scatter(x=data.diasem_hechos, y=data._id,
))
fig.update_layout(title=title)

fig.show()

In [34]:
title = 'Incidencia de delitos por dia de la semana desde el año 2010'
labels = delito_group["delito"].unique()

data = datos

fig = go.Figure()

for i in range(len(delito_group["delito"].unique())):
    datai=data[data.delito==labels[i]].groupby("diasem_hechos",as_index=False).count().sort_values(by="diasem_hechos")
    fig.add_trace(go.Scatter(x=datai.diasem_hechos, y=datai._id,
        name=labels[i],
        visible = "legendonly"
    ))
fig.update_layout(title=title)

fig.show()

### COMENTARIO:

Hay dos patrones principales: 

* Aumento de algunos crimenes los fines de semana (lesiones personales, robo a casas, violencia familiar)

* Disminucion de algunos crímenes los fines de semana (robos).

Estas diferencias se deben a la naturaleza propia de cada crimen, por lo que es necesario tomar esto encuenta si se quiere añadir este factor a algun tipo de algoritmo predictivo.

## Resultado:
Existe una estacional en los datos en los meses del año, siendo enero, marzo, mayo, junio y septiembre los meses con los maximos para delitos. 
Se observa la misma tendencia si miramos la curva para cada delito.
También se encontró una tendencia importante en los dias de la semana, que aunque no es estacional, puede servir para determinar que dias de la semana se presentan mas delitos de cierto tipo. Por ejemplo, los robos a la habitación sin violencia, suceden mas frecuentemente los fines de semana, se puede inferir que esto sucede cuando las casas se quedan solas porque sus dueños se van de paseo (los dias libres). 

## Anomalía de delitos por alcaldia.

Se busca encontrar cuales son las alcaldias que presentan mayor y menor incidencia de delitos

In [49]:
alcaldia_delitos = datos.groupby(["delito","alcaldia_hechos"],as_index=False).count()[["alcaldia_hechos","delito","_id"]]
alcaldia_delitos.columns=["alcaldia_hechos","delito","N"]

In [65]:
{"N alcaldias": len(datos.alcaldia_hechos.unique())}

{'N alcaldias': 205}

In [73]:
top_10 = alcaldia_delitos.groupby("alcaldia_hechos",as_index=False).sum().sort_values(by="N",ascending=False)[0:10]

In [77]:
fig = px.pie(top_10, values='N', names='alcaldia_hechos',title='Top 10 de alcaldias con mayor incidencia de delitos')
fig.update_traces(textposition='inside', textinfo='label')
fig.show()

In [114]:
button = alcaldia_delitos.groupby("alcaldia_hechos",as_index=False).sum().sort_values(by="N", ascending=True)
button=button[button.N<=5]

In [116]:
fig = px.bar(button, x="N", y="alcaldia_hechos", orientation='h',title="Alcaldias con menos de 6 delitos")
fig.show()

## Alcaldias anómalas por número de delitos

Esta anomalía se obtiene calculando el numero de delítos de cada alcaldía por tipo de delito. Luego se compara el numero de delitos (de cada tipo) de esa alcaldia con la estadística general. Obteniendo las alcaldias que tienen un tipo anormal de delito. Por ejemplo, una elevada tasa de robos.

In [125]:
alcaldia_delitos = datos.groupby(["alcaldia_hechos","delito"],as_index=False).count()[["alcaldia_hechos","delito","_id"]]
alcaldia_delitos.columns = ["alcaldia_hechos","delito","N"]

# promedio y desviacion standad general para cada delito
mean=alcaldia_delitos.groupby("delito",as_index=False).mean()
std=alcaldia_delitos.groupby("delito",as_index=False).std().fillna(0)

# estadistica de delitos
alcaldia_delitos_stad = pd.DataFrame({"delito":mean.delito,"mean":mean.N,"std":std.N})

In [126]:
# une la estadistica con los datos anteriores
alcaldia_delitos = pd.merge(alcaldia_delitos,alcaldia_delitos_stad)

#calculo de anomalia
alcaldia_delitos["max_delitos"] = alcaldia_delitos["mean"] + np.multiply(2,alcaldia_delitos["std"])
alcaldia_delitos["anomalo"] = alcaldia_delitos.N>alcaldia_delitos["max_delitos"]
alcaldia_delitos["anomalia"] = (alcaldia_delitos.N-alcaldia_delitos["mean"])/alcaldia_delitos["std"]

# promedio de la latitud y longitud
lugar=datos[["alcaldia_hechos","latitud","longitud"]].dropna().groupby("alcaldia_hechos",as_index=False).mean()

# incluye la latitud y longitud de la alcaldia (promedio)
alcaldia_delitos = pd.merge(alcaldia_delitos[alcaldia_delitos.anomalo==True],lugar, on = "alcaldia_hechos", how="left")

# data resultante
alcaldia_delitos.head()

,alcaldia_hechos,delito,N,mean,std,max_delitos,anomalo,anomalia,latitud,longitud
0,BENITO JUAREZ,DENUNCIA DE HECHOS,1040,142.989247,437.00434,1016.997928,True,2.052636,19.381891,-99.161730
1,CUAUHTEMOC,DENUNCIA DE HECHOS,3016,142.989247,437.00434,1016.997928,True,6.574330,19.430182,-99.149844
2,GUSTAVO A MADERO,DENUNCIA DE HECHOS,1205,142.989247,437.00434,1016.997928,True,2.430206,19.491411,-99.115353
3,IZTAPALAPA,DENUNCIA DE HECHOS,1820,142.989247,437.00434,1016.997928,True,3.837515,19.352637,-99.061913
4,VENUSTIANO CARRANZA,DENUNCIA DE HECHOS,1123,142.989247,437.00434,1016.997928,True,2.242565,19.428559,-99.102220


## Alcaldías con número de delitos anómalos (agrupado por delito)

In [128]:
import plotly.express as px
import plotly.graph_objects as go

data=[go.Bar(
    name=alcaldia,
    y=alcaldia_delitos[alcaldia_delitos.alcaldia_hechos==alcaldia].delito,
    x=alcaldia_delitos[alcaldia_delitos.alcaldia_hechos==alcaldia].anomalia,
    visible = "legendonly",
    orientation='h') for alcaldia in alcaldia_delitos.alcaldia_hechos.unique()]

fig = go.Figure(data=data)
# Change the bar mode

fig.update_layout(
    barmode='group',
    title='Alcaldias anomalas y sus delitos',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Delito',
        titlefont_size=16,
        tickfont_size=14,
    ),
    xaxis=dict(
        title='Anomalía',
        titlefont_size=16,
        tickfont_size=14,
    ),
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

## Estudio geográfico de los delitos

Tomaremos los años 2018 y 2019 para el estudio de anomalías por zonas, ya que se quiere observar el comportamiento de datos mas actualizados. Con los datos del número de crimenes agrupados por latitud-longitud, entrenaremos un algoritmo de machine learning basado en la densidad de puntos DBSCAN. Este algoritmo entrenado con 3 dimensiones (latitud, longitud y N crimenes) nos permitirá encontrar anomalias geo-espaciales en los datos 

In [157]:
indicador = datos[datos.delito.isin(delitos.delito[0:30])]
indicador = indicador[indicador.año_hechos>=2018]
indicador = indicador.groupby(["latitud","longitud"],as_index=False).count()[["latitud","longitud","_id"]].sort_values(by="_id",ascending=False)
indicador.columns=["latitud","longitud","N"]
indicador.head()

,latitud,longitud,N
1489,19.307951,-99.196828,10
4872,19.380237,-99.141160,8
5624,19.393029,-99.070157,8
8766,19.450194,-99.217653,7
3471,19.357818,-99.054345,6


In [158]:
from sklearn.cluster import DBSCAN
from sklearn import preprocessing

indicador_norm = preprocessing.StandardScaler().fit_transform(indicador)
clustering = DBSCAN(eps=3, min_samples=50).fit_predict(indicador_norm)

In [159]:
pred=clustering
indicador["pred"]=pred
indicador

,latitud,longitud,N,pred
1489,19.307951,-99.196828,10,-1
4872,19.380237,-99.141160,8,-1
5624,19.393029,-99.070157,8,-1
8766,19.450194,-99.217653,7,-1
3471,19.357818,-99.054345,6,-1
...,...,...,...,...
3613,19.360700,-99.234870,1,1
3614,19.360702,-99.247429,1,1
3615,19.360704,-99.189151,1,1
3616,19.360772,-99.189670,1,1


In [160]:
import plotly.express as px

fig = px.scatter_3d(indicador, x='longitud', y='latitud', z='N',symbol='pred',
              color='pred')
fig.show()
        

### COMENTARIO:
Los puntos morados indican las zonas con cantidad anormal de delitos

## Zonas con mayor número de delitos

In [162]:
indicador = pd.merge(indicador[["latitud","longitud","pred"]],datos,on=["latitud","longitud"])[["latitud","longitud","pred","calle_hechos","calle_hechos2","colonia_hechos","alcaldia_hechos"]]

In [163]:
result=indicador[indicador.pred==-1].groupby(["alcaldia_hechos","colonia_hechos","calle_hechos","calle_hechos2"],as_index=False).count()[["alcaldia_hechos","colonia_hechos","calle_hechos","calle_hechos2","pred"]].sort_values(by="pred",ascending=False)
result.columns=["alcaldia_hechos","colonia_hechos","calle_hechos","calle_hechos2","N"]
result.head()

,alcaldia_hechos,colonia_hechos,calle_hechos,calle_hechos2,N
20,GUSTAVO A MADERO,GUADALUPE TEPEYAC,CALZADA DE GUADALUPE,HENRY FORD,3
18,GUSTAVO A MADERO,ESTANZUELA,GENERAL GUADALUPE VICTORIA,LAS ARTES,2
28,MIGUEL HIDALGO,IRRIGACIÓN,MIGUEL DE CERVANTES SAAVEDRA,PRESA LA ANGOSTURA,1
21,IZTACALCO,AGRÍCOLA ORIENTAL,CALLE 245,ENTRE SUR 4 D,1
22,IZTACALCO,AGRÍCOLA ORIENTAL,CALZADA IGNACIO ZARAGOZA,PERIFERICO,1


## Las alcaldias anómalas

In [164]:
a = result.groupby("alcaldia_hechos",as_index=False).sum()[["alcaldia_hechos","N"]].sort_values(by="N", ascending=False)

fig = px.bar(a, x="N", y="alcaldia_hechos", orientation='h',title="Alcaldias con número anómalo de delitos")
fig.show()

## Colonias anómalas

In [165]:
a = result.groupby("colonia_hechos",as_index=False).count()[["colonia_hechos","N"]].sort_values(by="N", ascending=False)

fig = px.bar(a, x="N", y="colonia_hechos", orientation='h',title="Colonia con anomalía en número de delitos")
fig.show()

## Calles anómalas

In [166]:
a = result.groupby(["calle_hechos"],as_index=False).count()[["calle_hechos","N"]]

fig = px.bar(a, x="N", y="calle_hechos", orientation='h',title="Calles con número anómalo de delitos")
fig.show()

# Conclusion

* Se pudo observar que los delitos son estacionales para los meses del año, y que presenta un comportamiento característico para los dias de la semana dependiendo de la naturaleza del delito. No se encontró un comportamiento estacionario para los dias del mes.
* Se encontraron algunos datos perdidos hasta del 100 porciento. Datos que tengas muchos datos perdidos son difíciles de usar.
* Se encontró que hay 30 delitos que constituyen casi el 80% del total de delitos denunciados. 
* Los datos muestran que alrededor del año 2015 los delitos subieron extraordinariamente, mostrando un pico de delitos enorme para la ciudad. Sin embargo hubo una reducción significativa en los últimos años, el pico ha sido superado, y la ciudad muestra niveles de delito similares a los anteriores al pico. Además la tendencia es a seguir disminuyendo.

* Se hizo un estudio del número de delitos por alcaldía, y comparando los numeros de cada una con la estadistica general de crimenes, se logró aislar las alcaldias que tienen numerós anormales para ciertos delitos. Esto permitirá crear estrategias específicas para cada una de las alcaldías y atacar el fenómeno.

* Mediante un algoritmo de machine learning para agrupación de datos, se aislaron las zonas geográficas con número anormal de delitos. Esto se hizo con los datos de los dos últimos años, para asi tener información actualizada y poder observar las zonas mas problemáticas de la ciudad. Este algoritmo permite encontrar las areas más problematicas y gracias a esto desarrollar estrategias para abordar el problema.